In [ ]:
MYSQL INTRODUCTION 
=====================================

-> Mysql is a database management system.
-> Database is a structured collection of data.
-> To add , access and process data in a computer database , we need a database management system such as Oracle 
   , Mysql.

-> Mysql database is relational . 
-> A relational database stores data in seperate tables rather than putting all in one big file.
-> Rules are defined between different tables with related fields.

-> Mysql is open source . Mysql can be downloaded from internet and can be used without paying.
-> Mysql server is fast , reliable , scalable and easy to use.

In [ ]:
MYSQL VERSIONS 
===========================================

-> bugfix series like Mysql 8.4 
-> innovation release like Mysql 8.3

-> both relase contains bug fixes , however innovation release includes new feature.
-> both are meant for production use.


MYSQL 8.0.34 
-> 8 is the major version.
-> 0 is the minor version.
-> 34 is the bugfix release version.

In [ ]:
INSTALLING MYSQL ON LINIX USING GENERIC BINARIES
=============================================================


-> Download the TAR binary from the MySQL Downloads page, at https://dev.mysql.com/downloads/mysql/, 
   then choose Linux - Generic as the OS and select the version. You can download directly onto your 
   server  directly using the wget command:

  wget "https://downloads.mysql.com/archives/get/p/23/file/mysql-8.0.20-linux-glibc2.12-x86_64.tar.xz"


-> MySQL has a dependency on the libaio library. The data directory initialization, and subsequent server 
   startup steps, will fail if this library is not installed locally.

   yum install -y libaio


-> The OS 8 platforms by default do not install the file /lib64/libtinfo.so.5, which is required by the
   MySQL client  bin/mysql , To work around this issue, install the ncurses-compat-libs package.

   yum install ncurses-compat-libs


-> Add the mysql group and the mysql user. All the files and directories should be under the mysql user

   groupadd mysql
   useradd -r -g mysql -m-s /bin/false mysql
   cd  /home/mysql
   tar -zxvf mysql-8.0.20-linux-glibc2.12-x86_64.tar.xz
   cd /usr/local
   ln -s /home/mysql/mysql-8.0.20-linux-glibc2.12-x86_64 mysql  # make symbolic link
   export PATH=$PATH:/usr/local/mysql/bin


-> Create seperate directories for data , redolog , binlog , errorlog and systemdata 

   mkdir -p /opt/mysql/data
   mkdir -p /opt/mysql/redolog
   mkdir -p /opt/mysql/undolog
   mkdir -p /opt/mysql/systemdata
   mkdir -p /opt/mysql/mysql-files
 
   chown -R mysql:mysql /opt/mysql/
   chmod 750 /opt/mysql/mysql-files


-> Start mysqld by specfying datadir and user mysql 

   mysqld --initialize --datadir=/opt/mysql/data --user=mysql

   mysql_ssl_rsa_setup --datadir=/opt/mysql/data/

   mysqld_safe --user=mysql --datadir=/opt/mysql/data &

   mysql_secure_installation 


-> Copy startup files to init.d for auto start and update the datadir in the file

   cp support-files/mysql.server /etc/init.d/mysql.server

   vi /etc/init.d/mysql.server  -- datadir=/opt/mysql/data

   /etc/init.d/mysql.server stop
   /etc/init.d/mysql.server start


-> Create option/configuration file by adding different options

   vi /etc/mysql/my.cnf

    [mysqld]

    innodb-data-file-path = ibdata1:12M:autoextend
    innodb-data-home-dir = /opt/mysql/systemdata/

    innodb-undo-directory = /opt/mysql/undolog

    innodb-log-group-home-dir = /opt/mysql/redolog

    innodb-temp-data-file-path = /opt/mysql/tempdata/ibtmp1:12M:autoextend

    log-bin = /opt/mysql/binlog/prod01_binlog
    log-bin-index = /opt/mysql/binlog/prod01_binlog.index

    log-error = /opt/mysql/errorlog/prod01_mysqld.log

    server-id = 1
    

-> move files to respective directories and start the mysqld 

    [root@mysqlprod01 data]# mv ibdata1 ../systemdata/
    [root@mysqlprod01 data]# mv ib_logfile* ../redolog/

    mysqld_safe --user=mysql --datadir=/opt/mysql/data --syslog &

    mysql> select @@hostname , @@version;

In [ ]:
Installing MySQL on Linux Using the MySQL Yum Repository
 =======================================================================

 https://dev.mysql.com/downloads/repo/yum/

 
 -> install the repo locally and check /etc/yum.repos.d/ directory

     yum localinstall mysql84-community-release-el8-1.noarch.rpm

 -> check which version is enabled

    yum repolist enabled | grep mysql.*-community

 -> if you want to install 8.0 , then disable 8.4 and enable 8.0

    yum-config-manager --disable mysql-8.4-lts-community

    dnf config-manager --enable  mysql80-community  ( dnf is a newer tool similar to yum )

 -> install 8.0 community server

    yum install mysql-community-server


 -> start MySql service and enable auto-start 

      systemctl start mysqld
      systemctl enable mysqld.service

 -> change password of root user 

    grep 'temporary password' /var/log/mysqld.log
    mysql -u root -p


 -> verify whether mysql is running

    systemctl status mysqld
    pidof mysqld
    netstat -ntlp | grep 3306
    lsof -u mysql   ( list open files by user mysql)

In [ ]:
Installing MySQL on Linux Using  RPM Packages
=======================================================================

-> download locally the RPM packages from mysql site.

     mysql-community-server-8.0.20-1.el8.x86_64.rpm
     mysql-community-client-8.0.20-1.el8.x86_64.rpm
     mysql-community-libs-8.0.20-1.el8.x86_64.rpm
     mysql-community-devel-8.0.20-1.el8.x86_64.rpm
     mysql-community-common-8.0.20-1.el8.x86_64.rpm
     mysql-community-icu-data-files-8.0.20-1.el8.x86_64.rpm

-> install the packages individually or all at once.

    yum localinstall mysql-community-*.rpm

-> check the installed packages and start the server

    rpm -qa | grep mysql
    systemctl start mysqld
    systemctl enable mysqld

In [ ]:
Installing MySQL on Linux With Dokcer Containers
=======================================================================

-> add users to docker group 

   usermod -a -G docker mysqluser
   id mysqluser

-> search and pull mysql image 

   docker images
   docker ps
   docker search mysql
   docker pull mysql   ( for latest )
   docker pull mysql/mysql-server:8.0.30    ( for specific version )
   docker images

-> run mysql container 

   docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d mysql/mysql-server:8.0.30

   dokcer ps
   docker logs <container_name>


-> go into docker mysql host

   docker exec -it <container_name> bash 
   mysql -u root -p
   select @@hostname , @@version


->  run mysql container exposing the port 3306 from container to host. 
    by this outside client can be connected to mysql which is running in the container.

    docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d -p 3306:3306 mysql/mysql-server:8.0.30
               
-> run mysql container by binding local volume to container volume.
   by this even if the docker container is removed the local volume will not be destroyed.
   you can start new mysql container with this data.

   docker run --name mysql_8030 -e MYSQL_ROOT_PASSWORD=Root@2024 -d -p 3306:3306
   -v /var/lib/mysql:/var/lib/mysql mysql/mysql-server:8.0.30

In [ ]:
MySQL Upgrades 
===============================

Minor version upgarde -  8.0.28 -> 8.0.30  , in-place

Major version upgrade -  5.7 -> 8.0 , remove old and install new

-> Upgrade from 5.7 to 8.0 is suppoerted.
-> Upgrade is supported only between General Availabilty (GA).
-> Skipping a release is also supported ( 8.0.21 -> 8.0.23).


Upgrade Types 
===================

-> in-place upgrade - replacing the mysql server packages.
-> logical upgrade - exporting sql from old mysql instance into new.
-> replication topology upgrade.


Parts upgraded during the upgrade 
===========================================

-> Data dictionary upgrade - data dictionary tables in mysql , performance_schema , information_schema
                             and ndbinfo.

-> Server upgrade - system tables in mysql , sys schema and user schemas.

-> prior to 8.0.16 an extra step of mysql_upgrade has to be run by DBA.

-> After 8.0.16 the server itself performs all the tasks handled by mysql_upgrade.

In [ ]:
Minor Version Upgrade From 8.0.30 -> 8.0.35 ( rpm pacakges )
===============================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

   mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.35 --output-format=JSON 
              --config-path=/etc/my.cnf

-> Download 8.0.35 rpm packages 

->  Stop the mysqld server and install the packages

     systemctl stop mysqld

     yum localinstall mysql-community-*

-> verify the installed pacakges 

   rpm -qa | grep mysql

->  Start the mysqld server and check 
  
     systemctl start mysqld
     systemctl status mysqld
     pidof mysqld
     netstat -ntlp | grep 3306


In [ ]:
Major version upgrade from 5.7 to 8.30 ( binary )
=================================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

    mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.30 
            --output-format=JSON --config-path=/etc/mysql/my.cnf

-> downlod MYSQL 8.0.30 version binary 

   tar -xvf mysql-8.0.30-linux-glibc2.12-x86_64.tar.xz

->  shutdown mysqld service , unlink old binary and link to new binary

     /etc/init.d/mysql.server stop
     cd /usr/local
     unlink mysql
     ln -s /home/mysql/mysql-8.0.30-linux-glibc2.12-x86_64 mysql

-> start mysql , check and verify

   /etc/init.d/mysql.server start
   pidof mysqld
   netstat -ntlp | grep 3306

In [ ]:
Major version upgrade from 5.7 to 8.0.10 ( rpm packages )
=================================================================

-> check issues in the current database objects if any 

     mysqlcheck --all-databases --check-upgrade

-> check the target version is valid for upgrade

    mysqlsh -- util check-for-server-upgrade root@localhost --target-version=8.0.10 
            --output-format=JSON --config-path=/etc/mysql/my.cnf

-> downlod MYSQL 8.0.10 rpm packages 

->  shutdown mysqld service , and remove 5.7 packages

    systemctl stop mysqld
    pidof mysqld

    yum remove mysql-community-server-5.7.20-1.el7.x86_64
    yum remove mysql-community-common-5.7.20-1.el7.x86_64

-> install new 8.0.10 packages

   yum localinstall mysql-community-*

   rpm -qa | grep mysql

-> start mysql service and run upgrade ( prior to 8.0.16 )

   systemctl start mysqld

   mysql_upgrade

   select @@hostname , @@version;


In [ ]:
Downgrading MySQL 
=======================================

-> Downgrade from MySQL 8.0 to MySQL 5.7 is not supported.

-> Downgrade from a MySQL 8.0 release to a previous MySQL 8.0 release is also not supported.

-> Only supported alternative is to restore from a backup taken before upgrade.


In [ ]:
MySQL PROGRAMS 
================================

mysqld - is the main server program that does most of work in mysql installation.

mysqld_safe - is the server startup script and starts mysqld.

mysql.server - is also a server startup script . this invokes mysqld_safe to start the mysql server.

mysqld_multi - is a program that helps to start multiple mysql servers.

mysql_secure_installation - is a program that helps in securing mysql server.

mysql_ssl_rsa_setup - creates ssl certificates and key files for secure connections.

mysql_tzinfo_to_sql - this program loads the time zone tables in the mysql database from the 
                      host zoneinfo database.

mysql_upgrade - prior to 8.0.16 this program is used during upgrade after the new version is installed.

mysql - command line tool to access the mysql server.

mysqladmin - program used for administrative tasks.

mysqlcheck - program is used to repair , analyse and optimise tables.

mysqldump - program is used to dump mysql database into a file such as sql , text or xml.

mysqlimport - program is used to import text files into tables.

mysqlpump - similar to mysqldump with extra features.

mysqlsh - is a command line tool with extra scripting abilites like javascript and python.

mysqlshow - displays information about databases , tables ,columns and indexes.

mysqlslap - load emulation client. 

mysql_config_editor - used to store credentials in secure and encrypted form.

mysqlbinlog - used to read statements from binlog. this can be used during crash recovery and PITR.

mysqldumpslow - used to summarize the contents of slow query log.


In [ ]:

MySQL Config Editor 
=============================

-- helps in storing encrypted database credentials.
-- with filname .mylogin.cnf in users home directory.
-- default group is [client] and for any other group --login-path has to be specicified.

mysql_config_editor set -u root -p

mysql_config_editor print

mysql_config_editor print --all

mysql_config_editor set --login-path=dbadmin_client -u db_admin -p

mysql_config_editor reset


mysql_tzinfo_to_sql program
=======================================

-- loads time zone data from zoneinfo database into mysql database.
-- location in linux is /usr/share/zoneinfo

mysql_tzinfo_to_sql /usr/share/zoneinfo/ | mysql mysql


mysqlshow program
================================

-- displays database , table and column information.

mysqlshow employeesdb

mysqlshow employeesdb employees

mysqlshow employeesdb employees emp_id


mysqlcheck program
=======================================

-- is a table maintenence program , it checks , repairs , oprtimize or analyse the table.
-- table will be locked during the opertion.
-- -c check , -a analyze , -o optimize , -r repair.
-- -c is default.

mysqlcheck employeesdb employees

mysqlcheck -o -a employeesdb employees

mysqlcheck -r employeesdb employees


mysqlimport program
===================================

-- is a data import program , takes .txt file as input with tab delimited and table name as filename.
-- can be loaded only with location of secure_file_priv variable.

mysqlimport employeesdb staff_data.txt


MySQL Executing SQL Files 
==========================================

mysql> source create_table_staff.sql

mysql> \. create_table_staff.sql

mysql < create_table_staff.sql 

[mysqluser@mysqlserver ~]$ cat create_table_staff.sh
mysql < $1

[mysqluser@mysqlserver ~]$ ./create_table_staff.sh create_table_staff.sql &


Mysqladmin Program
===================================

-- is a program to carry out admin tasks like shutdown , create/drop database , check status , ping , 
   start/stop replica.

mysqladmin ping

mysqladmin status

mysqladmin shutdown

mysqladmin create employeesdb